In [ ]:
# import xarray as xr
# import pymannkendall as mk
# from tqdm import tqdm
# import numpy as np
# import os

# save_dir = "time_range_trend"
# os.makedirs(save_dir, exist_ok=True)
# time_range = slice("2000", "2020")
# ds = xr.open_dataset("data/amsmqtp_ssmi.nc")
# ds = ds.sel(time=time_range)
# ds = ds.resample(time="1Y").mean()
# lon = ds.lon.values
# lat = ds.lat.values
# time = ds.time.values
# data = ds.ssmi.values
# layer = ds.layer.values
# T, L, H, W = data.shape
# assert L == 5
# data = data.reshape(T, L * H * W)

# slopes = []
# pvalues = []
# for i in tqdm(range(L * H * W)):
#     res = mk.original_test(data[:, i])
#     p = res.p
#     slope = res.slope

#     pvalues.append(p)
#     slopes.append(slope)

# slopes = np.array(slopes).reshape(L, H, W)
# pvalues = np.array(pvalues).reshape(L, H, W)

# ds = xr.Dataset(
#         data_vars=dict(
#             slope=(["layer", "lat", "lon"], slopes), 
#             p=(["layer", "lat", "lon"], pvalues)
#         ), 
#         coords=dict(
#             lat=lat, lon=lon, layer=layer
#         )
#     )
# ds.to_netcdf(os.path.join(save_dir, "history_ssmi.nc"))

In [2]:
import xarray as xr
import pymannkendall as mk
from tqdm import tqdm
import numpy as np
import os
import pandas as pd

save_dir = "time_range_trend"
os.makedirs(save_dir, exist_ok=True)
season_map = {
    "MAM": [3, 4, 5], 
    "JJA": [6, 7, 8], 
    "SON": [9, 10, 11], 
    "DJF": [12, 1, 2]
}
time_range = slice("2000", "2020")

for k, v in season_map.items():
    ds = xr.open_dataset(f"data/amsmqtp_ssmi.nc")
    ds = ds.sel(time=time_range)
    ds = ds.sel(time=ds["time.month"].isin(v))
    if k == "DJF":
        ds["time"] = pd.to_datetime(ds["time"].values) - pd.DateOffset(months=2)
    ds = ds.resample(time="1Y").mean()
    lon = ds.lon.values
    lat = ds.lat.values
    time = ds.time.values
    data = ds.ssmi.values
    layer = ds.layer.values
    T, L, H, W = data.shape
    data = data.reshape(T, L * H * W)

    slopes = []
    pvalues = []
    for i in tqdm(range(L * H * W)):
        res = mk.original_test(data[:, i])
        p = res.p
        slope = res.slope

        pvalues.append(p)
        slopes.append(slope)

    slopes = np.array(slopes).reshape(L, H, W)
    pvalues = np.array(pvalues).reshape(L, H, W)

    ds = xr.Dataset(
            data_vars=dict(
                slope=(["layer", "lat", "lon"], slopes), 
                p=(["layer", "lat", "lon"], pvalues)
            ), 
            coords=dict(
                lat=lat, lon=lon, layer=layer
            )
        )
    ds.to_netcdf(os.path.join(save_dir, f"history_{k}_ssmi.nc"))

/data/home/scv7343/.conda/envs/climsim/lib/python3.10/site-packages/xarray/core/groupby.py:668: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  index_grouper = pd.Grouper(
100%|██████████| 53605/53605 [01:05<00:00, 815.02it/s]
/data/home/scv7343/.conda/envs/climsim/lib/python3.10/site-packages/xarray/core/groupby.py:668: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  index_grouper = pd.Grouper(
100%|██████████| 53605/53605 [01:04<00:00, 834.83it/s]
/data/home/scv7343/.conda/envs/climsim/lib/python3.10/site-packages/xarray/core/groupby.py:668: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  index_grouper = pd.Grouper(
100%|██████████| 53605/53605 [01:10<00:00, 758.93it/s]
/data/home/scv7343/.conda/envs/climsim/lib/python3.10/site-packages/xarray/core/groupby.py:668: FutureWarning: 'Y' is deprecated and will be removed in a future